In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

In [2]:
batch_size = 128
steps = 200*1000
# tensorboard_writer = SummaryWriter()
T_0 =3
T_mult=2
eta_min = 1e-6
epsilon = 0.01
matrix_size=16
base_matrix = np.load('base_matrix.npy')
inv_base_matrix = np.load('inv_base_matrix.npy')

In [3]:


class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.fc1 = nn.Linear(256,2048)
        self.fc3 = nn.Linear(2048, 256)
        
    def forward(self, x):
        x = x.view(-1, 256)   # reshape Variable
        x = F.relu(self.fc1(x))
#         x = F.dropout(x, 0.1)
        x = self.fc3(x)
        return x
    
model = BaseModel()
model = model.to(torch.double)
model = model.to('cuda') 
model.train()
model

BaseModel(
  (fc1): Linear(in_features=256, out_features=2048, bias=True)
  (fc3): Linear(in_features=2048, out_features=256, bias=True)
)

In [4]:
class CustomDataset(Dataset):

    def __init__(self, num_per_epoch, batch_size):
        self.num_per_epoch = num_per_epoch
        self.batch_size = batch_size

    def __len__(self):
        return self.num_per_epoch

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        x = np.zeros((matrix_size,matrix_size), dtype = np.float64)
        y = np.zeros((matrix_size,matrix_size), dtype = np.float64)
        
        temp = np.random.rand(matrix_size,matrix_size).astype(np.float64)
        temp = temp*2-1
        cur_matrix = base_matrix + epsilon*temp
        temp_inv = np.linalg.inv(cur_matrix)
    
        x[:,:] = temp[:,:]
        y[:,:] = temp_inv[:,:]
        return x,y

In [5]:
train_set = CustomDataset(100000,batch_size)
val_set = CustomDataset(10000,batch_size)
train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(
    val_set,
    batch_size=batch_size)

In [6]:
optimizer = optim.Adam(model.parameters(), lr=5e-5, weight_decay=1e-7)
scheduler = CosineAnnealingWarmRestarts(optimizer,T_0,T_mult,eta_min)

In [7]:

train_accu = []
i = 1
for epoch in range(steps//len(train_loader)):
    for data, target in train_loader:
#         target = target - torch.tensor(inv_base_matrix)
#         print(target)
#         target = target/epsilon
        data, target = Variable(data), Variable(target)
        data = data.to('cuda')
        target = target.to('cuda')
        optimizer.zero_grad()
        output = model(data)
        loss = F.mse_loss(output, target.view(-1,256))
        loss.backward()
        
        mse_loss = loss.item()
        optimizer.step()
        
#         if i % 10 == 0:
#             tensorboard_writer.add_scalar("Loss/step", loss, i)
        if i % 100 == 0:
            print('\rTrain Step: %d, Loss: %.4f, lr: %.8f'%(i, mse_loss, scheduler.get_lr()[0]), end="")
        i += 1
    scheduler.step()
    print('\n')

C:\D_software\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:971: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Train Step: 700, Loss: 0.0021, lr: 0.00005000

Train Step: 1500, Loss: 0.0014, lr: 0.00003775

Train Step: 2300, Loss: 0.0011, lr: 0.00001325

Train Step: 3100, Loss: 0.0006, lr: 0.00005000

Train Step: 3900, Loss: 0.0003, lr: 0.00004672

Train Step: 4600, Loss: 0.0002, lr: 0.00003775

Train Step: 5400, Loss: 0.0001, lr: 0.00002550

Train Step: 6200, Loss: 0.0001, lr: 0.00001325

Train Step: 7000, Loss: 0.0001, lr: 0.00000428

Train Step: 7800, Loss: 0.0000, lr: 0.00005000

Train Step: 8600, Loss: 0.0000, lr: 0.00004917

Train Step: 9300, Loss: 0.0000, lr: 0.00004672

Train Step: 10100, Loss: 0.0000, lr: 0.00004282

Train Step: 10900, Loss: 0.0000, lr: 0.00003775

Train Step: 11700, Loss: 0.0000, lr: 0.00003184

Train Step: 12500, Loss: 0.0000, lr: 0.00002550

Train Step: 13200, Loss: 0.0000, lr: 0.00001916

Train Step: 14000, Loss: 0.0000, lr: 0.00001325

Train Step: 14800, Loss: 0.0000, lr: 0.00000818

Train Step: 15600, Loss: 0.0000, lr: 0.00000428

Train Step: 16400, Loss: 0.0000, 

In [8]:
torch.save(model.state_dict(), 'direct_3.pth')

In [8]:
from tqdm import tqdm
model.load_state_dict(torch.load('direct_3.pth'))
train_accu = 0
i = 1
model = model.eval()
total_error = 0.0
total_number = 0
for data, target in tqdm(val_loader):
    target = target - torch.tensor(inv_base_matrix)
#     target = target/epsilon
    data, target = Variable(data), Variable(target)
    data = data.to('cuda')
#     target = target.to('cuda')
    output = model(data)
    output = output.detach().to('cpu') - torch.tensor(inv_base_matrix).view(-1,256)
#     output = output/epsilon
    total_error += torch.sum(torch.abs(output[:,:] - target.view(-1,256)[:,:]))
    total_number += output.shape[0]*output.shape[1]
#     print(total_error/total_number)
#     print(output[:5,:])
#     print(target.view(-1,4)[:5,:])
#     break

print(total_error.numpy()/total_number)

100%|██████████| 79/79 [00:01<00:00, 48.80it/s]

0.00016585950074728394


####

model 1 test error: 0.00019151217204180958
model 2 test error: 0.00014824967359452205
model 3 test error: 0.00016441494669052606

In [10]:
temp = [0.00019151217204180958, 0.00014824967359452205, 0.00016441494669052606]
print(np.mean(temp))
print(np.var(temp))

0.00016805893077561924
3.185799386564655e-10


In [9]:
print(output[0,:10])
print(target[0,:10])

tensor([-0.0032,  0.0055,  0.0088,  0.0040, -0.0054,  0.0058,  0.0046,  0.0038,
        -0.0051, -0.0065], dtype=torch.float64)
tensor([[-3.1091e-03,  5.3981e-03,  8.8131e-03,  4.0846e-03, -5.5218e-03,
          5.7563e-03,  4.7135e-03,  3.8447e-03, -5.1913e-03, -6.5363e-03,
          1.9280e-03, -2.8761e-03,  6.0036e-03, -1.1107e-03, -2.7868e-03,
          2.1972e-03],
        [ 3.6838e-03, -1.5283e-03,  5.8650e-04,  1.9677e-03, -1.7887e-03,
          3.3022e-03, -7.3312e-04, -1.9070e-03, -2.4562e-03, -1.3982e-04,
          6.2427e-04,  3.6527e-03, -9.1838e-04, -1.5825e-03,  5.5166e-04,
          1.0555e-04],
        [-6.2596e-03,  3.3457e-03,  1.8593e-03, -1.4196e-03,  1.3211e-03,
         -3.1778e-03,  3.7739e-03,  4.6766e-03,  3.3859e-03, -3.4816e-03,
         -4.4908e-04, -5.1769e-03,  2.9744e-03,  1.4741e-03, -2.5929e-03,
         -1.8147e-03],
        [-2.1007e-03,  5.3185e-03,  2.8810e-03, -1.7979e-03, -2.0480e-03,
          2.3974e-03,  9.3983e-04,  1.2444e-03, -2.9042e-03, -1